# Rigid registration

This example should show you how to perform rigid registration between two SIRF images. It is recommended that you check the [Geometry notebooks](../Geometry) first.

SIRF's registration/resampling functionality is provided by wrapping and extending the [NiftyReg](http://cmictig.cs.ucl.ac.uk/wiki/index.php/NiftyReg) code base. Rigid and affine registrations are performed using NiftyReg's symmetric `aladin` algorithm, whereas non-rigid registrations use the symmetric `f3d` algorithm.

Although the example of rigid registration is given here, it is trivial to modify it for affine registrations and only slightly trickier to extend to non-rigid registration.

The images to be registered in this example are `test.nii.gz` and `test2.nii.gz`, which are two T1-weighted MR brain scans taken one year apart.

N.B.: Registration packages use different names for the sets of images they are registering. In NiftyReg (and therefore SIRF), the floating image is moved to match the reference image. In other packages, the floating=moving and reference=fixed.

N.B.: If you have MATLAB, you can install SPM. `SIRF` will then be able to wrap the `spm_realign` function as well (even from Python!). This is not illustrated here, but see the `sirf.Reg.SPMRegistration` class documentation.

## Copyright

Author: Richard Brown  
First version: 3rd April 2019  
Second version: 8th June 2020  

CCP PETMR Synergistic Image Reconstruction Framework (SIRF)  
Copyright 2019-2021 University College London.

This is software developed for the Collaborative Computational Project in Positron Emission Tomography and Magnetic Resonance imaging (http://www.ccppetmr.ac.uk/).

SPDX-License-Identifier: Apache-2.0


## Initial set up

In [ ]:
#%% make sure figures appears inline and animations works
%matplotlib widget

# Setup the working directory for the notebook
import notebook_setup
from sirf_exercises import cd_to_working_dir
cd_to_working_dir('Reg', 'Registration')

In [ ]:
# Standard stuff
import os
import numpy
import matplotlib.pyplot as plt

# SIRF stuff
from sirf.Utilities import examples_data_path
import sirf.Reg as Reg
examples_path = examples_data_path('Registration')

In [ ]:
#%% First define some handy function definitions
# To make subsequent code cleaner, we have a few functions here. You can
# ignore them when you first see this demo.
# They have (minimal) documentation using Python docstrings such that you 
# can do for instance "help(imshow)"
#
# First a function to display an image

def imshow(image, title=''):
    """Display an image with a colourbar, returning the plot handle. 

    Arguments:
    image -- a 2D array of numbers
    limits -- colourscale limits as [min,max]. An empty [] uses the full range
    title -- a string for the title of the plot (default "")
    """
    plt.title(title)
    bitmap=plt.imshow(image)
    limits=[numpy.nanmin(image),numpy.nanmax(image)]

    plt.clim(limits[0], limits[1])
    plt.colorbar(shrink=.6)
    plt.axis('off')
    return bitmap

## Engine for loading images

By default this example uses `sirf.Reg` as the engine to open the images, which handles NIfTI images. 

You might want to register different types of images - perhaps your floating image is a STIR interfile? If so, change the second line such that it reads `import sirf.STIR as eng_flo`.

In [ ]:
# In this example, we will use sirf.Reg as the engine to open our images
import sirf.Reg as eng_ref
import sirf.Reg as eng_flo

## Open and display the images

In [ ]:
# Open the images
ref_file = os.path.join(examples_path, "test.nii.gz")
flo_file = os.path.join(examples_path, "test2.nii.gz")
ref = eng_ref.ImageData(ref_file)
flo = eng_flo.ImageData(flo_file)

In [ ]:
# Display the images
ref_slice = ref.get_dimensions()[1] // 2
flo_slice = flo.get_dimensions()[1] // 2
plt.subplot(1,2,1)
imshow(ref.as_array()[ref_slice,:,:], 'Reference image, slice: %i' % ref_slice)
plt.subplot(1,2,2)
imshow(flo.as_array()[flo_slice,:,:], 'Floating image, slice: %i' % flo_slice);

## A little word about image orientation

An image will needs information in order to be able to place it in the real world. This information encodes things such as voxel sizes, orientation and offset or origin (where the image starts). The geometrical information for any SIRF image can be extracted with `get_geometrical_info`.

This is discussed in detail in the [Geometry notebooks](../Geometry).

Let's have a look at the direction matrix and offset for our two example images:

In [ ]:
# Ref
ref_geom_info = ref.get_geometrical_info()
ref_direction_matrix = ref_geom_info.get_direction_matrix()
print("printing ref direction matrix\n", ref_direction_matrix)
ref_offset = ref_geom_info.get_offset()
print("printing ref offset\n", ref_offset)
# Flo
flo_geom_info = flo.get_geometrical_info()
flo_direction_matrix = flo_geom_info.get_direction_matrix()
print("\nprinting flo direction matrix\n", flo_direction_matrix)
flo_offset = flo_geom_info.get_offset()
print("printing flo offset\n", flo_offset)

This means that our `imshow` above is misleading. It uses `as_array`, which returns the underlying data "as is", without any regard for the image orientation. Luckily, the registration and resampling tools understand this and will take it automatically into account.

Hopefully you can convince yourself that, in spite of the misleading orientation of the first `imshow`, the images are actually facing the same direction. 

Now have a look at our offsets that we printed further up. These are quite different. We would therefore expect the rigid registration between these two images would result in little rotation, but quite a lot of translation. 

Let's have a look!

## Back to registration!

Ok, let's get back to the registration. Start by setting up the registration object.

In [ ]:
# Set to NiftyF3dSym for non-rigid
algo = Reg.NiftyAladinSym()

# Set images
algo.set_reference_image(ref)
algo.set_floating_image(flo)

# What else can we do?
help(algo)

## Setting parameters

From the help above, it looks like we can set registration parameters both via a file and directly. Let's try both.

In [ ]:
# Setting via parameter file
par_file = os.path.join(examples_path, "paramFiles", "niftyreg_aladin.par")
algo.set_parameter_file(par_file)

algo.set_parameter('SetPerformRigid','1')
algo.set_parameter('SetPerformAffine','0')
#algo.set_parameter('SetWarpedPaddingValue','0')  # NaN by default, uncomment to set to 0.

## Register!

In [ ]:
algo.process()

## Show the registered image

The registered image will be the same size as the reference image, so we can use `ref_slice`.

In [ ]:
output = algo.get_output()
output_arr = output.as_array()
plt.figure()
imshow(output_arr[ref_slice,:,:], 'Registered image, slice: %i' % ref_slice);

## Show the transformation matrix

In [ ]:
numpy.set_printoptions(precision=3,suppress=True)
TM = algo.get_transformation_matrix_forward()
print(TM.as_array())

So as we predicted earlier, there is little rotation (close to identity in the top 3x3 of the matrix), and there is large translation (final column).

## Displacement field image
The displacement field image maps the voxels of the floating image to the final warped image. They are particularly interesting for non-rigid registrations, where transformation matrices no longer exist.

Each voxel of the displacement field image contains an (x,y,z) coordinate, so the resulting image is 4D.

(As a small technicality, the NIfTI format stores the time component in the 4th dimension, and the displacement coordinates in the 5th dimension. Therefore the displacement field image is actually a 5D image with a singleton in the 4th dimension.)

***WARNING***: the displacement vectors from `sirf.Reg` are currently (SIRF 3.1) in RAS coordinates as used by NIfTI.

In [ ]:
displacement = algo.get_displacement_field_forward()

plt.figure()
for i in range(3):
    plt.subplot(3, 1, 1 + i)
    imshow(displacement.as_array()[ref_slice,:,:,0,i],
           'Displacement field %s-direction, slice: %i' % ("xyz"[i], ref_slice))

# Resampling

Imagine that you got the motion information from another source and wanted to resample an image with it. Easy!

We just need to create a resampler and set our image to resample as a floating image. As image domains can be different (i.e. images from different modalities, images with different field of view, etc), we also need to give the resampler a "template" image to define the resampled space. We can do this by setting the reference image. If we want to resample the image to the same domain as the original, you can set the floating image and the reference image to be the same.

We would also need to define the type of transformation. We can use transformation matrices here, but it can also be a displacement of a deformation field, if your application requires it. 

Finally, we can also define the type of interpolation to use. In this example, we use linear with `set_interpolation_type_to_linear()`, but we can also use `_nearest_neighbour()`, `_cubic_spline()` and `_sinc()` instead of `_linear()`. Or just set it with `set_interpolation_type(int)`, with their respective order of interpolation.




In [ ]:
tm = algo.get_transformation_matrix_forward()
# get the resampler!
resampler = Reg.NiftyResample()
# Make sure we know what the resampled image domain looks like (this can be the same as the image to resample)
resampler.set_reference_image(ref)
# Set image to resample
resampler.set_floating_image(flo)
# Add the desired transformation to apply
resampler.add_transformation(tm)
resampler.set_padding_value(0)
# Use linear interpolation
resampler.set_interpolation_type_to_linear()
# Go!
resampler.process()

plt.figure()
imshow(resampler.get_output().as_array()[ref_slice,:,:], 'Resampled image, slice: %i' % ref_slice);